<a href="https://colab.research.google.com/github/ccosmin97/IDC-Prostate_segmentation/blob/main/idc_prostate_segmentation_promised24_QIN_PROSTATE_Repeatability_EXPORT_OHIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prostate segmentation on IDC collection -- QIN-REP
*   Dataset : [QIN-REP](https://portal.imaging.datacommons.cancer.gov/explore/filters/?collection_id=qin_prostate_repeatability)
*   Goal : Prostate segmentation using D24 Promise nnUnet pre-trained model, one T2 modality input

# Auth login

##Colab

In [20]:
#colab 
from google.colab import auth
auth.authenticate_user()

## Drive setup for upload results

In [21]:
# import os
# #drive mount
# from google.colab import drive
# drive.mount('/content/drive')
# ##setup paths
# path_drive_root = "/content/drive/MyDrive/"
# drive_exp_root_name = "IDC" 
# data = "qin-prostate-repeatability"
# pre_trained_mod = "task024-promise"
# timestamp = "2022_07_11"#year_month_day

In [22]:
# #personal drive path -- make sure folder exists in google drive
# path_experiment = os.path.join(path_drive_root,
#                                drive_exp_root_name,
#                                data,
#                                pre_trained_mod,
#                                timestamp)

# if not os.path.exists(path_experiment):
#   !mkdir -p $path_experiment

# Global variables

## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [23]:
# IDC collection for paths setup
collection_name = "qin_prostate_root_data"
##nnunet
model_type = '3d_fullres'#other options are '2d',..
##labelID seg retrieval
ground_truth_cat_CodeMeaning = 'Anatomical Structure'
ground_truth_type_CodeMeaning = 'Prostate'

## Global paths

In [117]:
import os 

#qin_prostate_repeatibility
os.environ["qin_prostate_rep_root"] = os.path.join(os.getcwd(), collection_name)
os.environ["qin_prostate_rep_dicom"] = os.path.join(os.environ["qin_prostate_rep_root"], "dicom")
os.environ["qin_prostate_rep_nii"] = os.path.join(os.environ["qin_prostate_rep_root"], "nii")
os.environ["qin_prostate_rep_nrrd"] = os.path.join(os.environ["qin_prostate_rep_root"], "nrrd")
os.environ["qin_prostate_rep_analysis"] = os.path.join(os.environ["qin_prostate_rep_root"], "results")
os.environ["qin_prostate_rep_analysis_verbose"] = os.path.join(os.environ["qin_prostate_rep_root"], "results_verbose")

#resampled data -- To t2 mod
os.environ["qin_prostate_rep_root_resampled"] = os.path.join(os.environ["qin_prostate_rep_root"], "resampled")
os.environ["qin_prostate_rep_nii_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nii")
os.environ["qin_prostate_rep_nrrd_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nrrd")

#nnunet
os.environ["nnUNet"] = os.path.join(os.getcwd(), "nnUNet")
os.environ["nnUNet_data"] = os.path.join(os.environ["nnUNet"], "data")
os.environ['nnUNet_raw_data_base'] = os.path.join(os.environ["nnUNet_data"], "nnUNet_raw_data")
os.environ['nnUNet_preprocessed'] = os.path.join(os.environ["nnUNet_data"], "processed")
os.environ["nnUNet_models"] = os.path.join(os.environ["nnUNet"], "models")
os.environ["RESULTS_FOLDER"] = os.path.join(os.environ["nnUNet"], "output", "preds")
os.environ["nnUNet_preds_resampled"] = os.path.join(os.environ["nnUNet"], "output", "preds_resampled")
os.environ["nnUNet_preds_resampled_dcm"] = os.path.join(os.environ["nnUNet"], "output", "preds_resampled_dcm")
os.environ["nnUNet_raw_data"] = os.path.join(os.environ["nnUNet"], "raw_data")
#path where model pre-trained weights are stored
os.environ["PATH_TO_MODEL_FILE"] = os.path.join(os.environ["nnUNet"], "models", "Task024_Promise.zip")

#misc
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")

#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["qin", "nnunet", "IDC", "nnUNet"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Setup GCP Project ID

In [25]:
project_id = "idc-nlst-unet-seg"
os.environ["GCP_PROJECT_ID"] = project_id

# Download nnUnet pre-trained model on prostate decathlon data -- task24 -- personal dropbox

nnUnet pre-trained models zip files can also be found [here](https://zenodo.org/record/4003545#.YsWmH-zMLt8).

In [26]:
print(os.environ["nnUNet_models"])

/content/nnUNet/models


In [27]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_model_url = "https://www.dropbox.com/s/u9m37l8et4hgu4h/Task024_Promise.zip?dl=0"
out_path_mod = os.path.join(os.environ["nnUNet_models"], "Task024_Promise.zip")#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $out_path_mod $seg_model_url

--2022-08-30 22:03:59--  https://www.dropbox.com/s/u9m37l8et4hgu4h/Task024_Promise.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/u9m37l8et4hgu4h/Task024_Promise.zip [following]
--2022-08-30 22:04:00--  https://www.dropbox.com/s/raw/u9m37l8et4hgu4h/Task024_Promise.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfa57857c314ff7ebdb50261040.dl.dropboxusercontent.com/cd/0/inline/BsCnPtFHC5pChUuUQJIHSWkaV_GyDOdM1CPpQCkDg7jwA7KyNM3YGXIRZSvEXLp_XVdWvQcHe7Zn0qFz1OOR3pLz2G6QUSv2ftcvK1N2vVLNynhwykTFCM9wVGghLBphOIjEcrKBi6299eNgyWnKA1-KEi5kKJBjrvyzlyD6MYrm0A/file# [following]
--2022-08-30 22:04:00--  https://ucfa57857c314ff7ebdb50261040.dl.dropboxusercontent.com/cd/0/inline/BsCnPtFHC5pChUuUQJIHSWkaV_GyDOdM1CPpQCkDg7jwA7KyNM3YGX

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies. 

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

In [28]:
# check wether the Colab Instance was correctly initialized with a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [29]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Tue Aug 30 22:04:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Environment Setup

Here we will configure the Linux environment variables needed to run the nnU-Net pipeline. 

Three main variables are needed by default to run the nnU-Net segmentation pipelines:
* `nnUNet_raw_data_base` is the path to the folder where the segmentation pipeline expects to find the data to process;
* `nnUNet_preprocessed` is the path to the folder where the preprocessed data are saved;
* `RESULTS_FOLDER` is the path to the folder storing by default the model weights and, in our case, for simplicity, the segmentation masks produced by the pipeline.

We will use the additional variable `PATH_TO_MODEL_FILE` to point to the location where the pre-trained model weights for the chosen model will be stored (more on this later).

Please notice that these variables need to be set using `os.environ[]` in Google Colab - as `!export` is not sufficient to guarantee the variables are kept from one cell to the other. For more in-depth information regarding what the nnU-Net framework uses these folders for, please visit [the dedicated nnU-Net documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md)

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [30]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [31]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [81]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [33]:
%%capture

!sudo apt install subversion

In [34]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled May 21 2022, 07:24:25 on x86_64-pc-linux-gnu


## Install Python packages

In [35]:
%%capture
!pip install nnunet
!pip install pydicom
!pip install nibabel

Unpack and install model we downloaded earlier (under `PATH_TO_MODEL_FILE`). This step can take about 1-2 minutes.

In [36]:
%%capture
!nnUNet_install_pretrained_model_from_zip $PATH_TO_MODEL_FILE

Next we set up few things to help with visualization of the segmentations later.

In [37]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

from medpy.metric.binary import dc as dice_coef
from medpy.metric.binary import hd as hausdorff_distance
from medpy.metric.binary import asd as avg_surf_distance

from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
%tensorflow_version 2.x
import tensorflow as tf
import keras

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)
print("TensorFlow version           : ", tf.__version__)
print("Keras (stand-alone) version  : ", keras.__version__)

print("\nThis Colab instance is equipped with a GPU.")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Python version               :  3.7.13 (default, Apr 24 2022, 01:04:09) 
Numpy version                :  1.21.6
TensorFlow version           :  2.8.2
Keras (stand-alone) version  :  2.8.0

This Colab instance is equipped with a GPU.


In [38]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

A    pyplastimatch/__init__.py
A    pyplastimatch/pyplastimatch.py
A    pyplastimatch/utils
A    pyplastimatch/utils/__init__.py
A    pyplastimatch/utils/data.py
A    pyplastimatch/utils/eval.py
A    pyplastimatch/utils/viz.py
Checked out revision 18.


In [39]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [40]:
from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.utils import viz as viz_utils
from pyplastimatch.utils import data as data_utils

# Data selection, downloading and structuring

We want to select here the collection named qin-prostate repeatibility, and more particularly the two timepoints per patient ID for further analysis.

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

## Data selection -- BigQuery

Explanation

In [41]:
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])
selection_query = f"""
  SELECT  
  StudyInstanceUID, 
  SeriesInstanceUID, 
  SOPInstanceUID, 
  PatientID, 
  Modality, 
  collection_id, 
  StudyDescription, 
  SeriesDescription, 
  StudyDate, 
  SeriesDate,
  gcs_url,
  (SELECT SeriesInstanceUID FROM UNNEST(ReferencedSeriesSequence)) AS RefSerieUID 
FROM 
  `bigquery-public-data.idc_current.dicom_all` 
WHERE 
  collection_id = 'qin_prostate_repeatability' 
  AND SeriesDescription IN ('T2 Weighted Axial', 'T2 Weighted Axial Segmentations')""" 

In [42]:
selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [100]:
selection_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 8 to 898
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   StudyInstanceUID   58 non-null     object
 1   SeriesInstanceUID  58 non-null     object
 2   SOPInstanceUID     58 non-null     object
 3   PatientID          58 non-null     object
 4   Modality           58 non-null     object
 5   collection_id      58 non-null     object
 6   StudyDescription   58 non-null     object
 7   SeriesDescription  58 non-null     object
 8   StudyDate          58 non-null     object
 9   SeriesDate         58 non-null     object
 10  gcs_url            58 non-null     object
 11  RefSerieUID        2 non-null      object
dtypes: object(12)
memory usage: 5.9+ KB


## Randomly pick 10 SutdyIDs for the use case

In [43]:
# #Random PatientID selection
# patientID_select = random.sample(list(selection_df.PatientID.values), 1)

# #Static PatientID selection
# patientID_select = ["PCAMPMRI-00014"]

# selection_df = selection_df[selection_df.PatientID.isin(patientID_select)]

# selection_df.PatientID.unique()

array(['PCAMPMRI-00014'], dtype=object)

## Download gs urls into dcm files and sorting  

In [44]:
# selection_df=pd.read_csv(selection_df)#"t2_and_seg.csv")
print(f"number of unique patientIDS that have both t2 axial and seg whole prostate objects : {len(sorted(selection_df.PatientID.unique()))}")

number of unique patientIDS that have both t2 axial and seg whole prostate objects : 1


In [45]:
# save the list of GCS URLs into a file
selection_manifest = os.path.join(os.environ["IDC_Downloads"], "idc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [46]:
# confirm the resulting manifest has as many lines as the number of rows in the
# dataframe we initialized earlier
!cat {selection_manifest} |wc

     58      58    3828


In [47]:
# let's make sure the download folder is clean, in case you ran this cell earlier
# for a different dataset
!rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}

In [48]:
# download is this simple!
%%capture

!cat {selection_manifest} | gsutil -m cp -I {os.environ["IDC_Downloads"]}

To facilitate preparation of the data, we will sort the downloaded files into the PatientID/StudyInstanceUID/SeriesInstanceUID hierarchy.

In [49]:
!python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 59/59 [00:00<00:00, 167.14it/s]
Files sorted


Move the sorted data into the right place

In [50]:
rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}* && mv {os.environ["IDC_Downloads_Sorted"]+"/*"} {os.environ["qin_prostate_rep_dicom"]}

#Pre-Processing

Dicom files are assumed to be sorted this way for further processing :

* PatientID
  * StudyUID
    * SeriesUID
      * *.dcm file(s)

## conversion methods

https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24283641

switch from plastimatch resample method to simple itk for slice spacing issue

In [51]:
def safe_sitk_read(img_list, *args, **kwargs):
  """
  Since the default function just looks at images 0 and 1 to determine slice thickness
  and the images are often not correctly alphabetically sorted, much slower
  :param img_list:
  :return:
  """
  pimg_list = [(sitk.ReadImage(x).GetOrigin(), x) for x in img_list]
  s_img_list = [path for _, path in sorted(pimg_list, key = lambda x: x[0][2])] # sort by z
  return sitk.ReadImage(s_img_list, *args, **kwargs)

In [52]:
def convert_mr_to_nii_nrrd(input_path, output_path_root, modality, target_format="nii.gz"):
  if not os.path.exists(output_path_root): 
    !mkdir -p $output_path_root
  output_path = os.path.join(output_path_root, input_path.split('/')[-1][:-4] 
                             + "_MR_" + modality
                             + "." + target_format)
  #convert to target format
  if not os.path.exists(output_path):
    dcm_img = safe_sitk_read(glob.glob(os.path.join(input_path, "**", "*.dcm"), recursive=True))
    sitk.WriteImage(dcm_img, output_path)

We will use the [segimage2itkimage tool](https://qiicr.gitbook.io/dcmqi-guide/opening/cmd_tools/seg/segimage2itkimage) from dcmqi to extract individual segments from DICOM SEG series we just downloaded into NII format, saving each segment into a separate file

In [53]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path): 
    !mkdir -p $output_path
  
  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii 

In [54]:
def convert_dcm_sorted(input_path,output_path):
  path_dicom = os.environ["qin_prostate_rep_dicom"] 
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#, recursive = True):
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    type_serie = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#'SEG' if pydicom.dcmread(path_serie_dcm_lst[0]).Modality == "SEG" else "MR"
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    if type_serie == "SEG":
      #convert to nrrd
      convert_seg_to_nii(input_path=path_serie_dcm_lst[0],
                              output_path=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_SEG"))
    elif type_serie == "MR":
      #convert to nii
      convert_mr_to_nii_nrrd(input_path=serie_folder,
                              output_path_root=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), modality="T2", target_format="nii.gz")
      #convert to nrrd
      convert_mr_to_nii_nrrd(input_path=serie_folder, 
                              output_path_root=os.path.join(output_path, "nrrd", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), modality="T2", target_format="nrrd")
    else:
      print(f"type serie {type_serie} not supported")

## Conversion to NII/NRRD

In [55]:
!rm -rf /content/qin_prostate_root_data/nii/*
!rm -rf /content/qin_prostate_root_data/nrrd/*

In [56]:
convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
                   output_path=os.environ["qin_prostate_rep_root"])

input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994/1.2.276.0.7230010.3.1.4.1426846371.11304.1513205200.995.dcm
output_path : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: ef9e227 tag: v1.2.4
W: CodeValue (0008,0100) empty in CodeSequenceMacro (type 1C)
Row direction: 0.999976 -0.00674836 -0.00175933
Col direction: 0.00690929 0.993212 0.116116
Z direction: 0.000963796 -0.116125 0.993234
Total frames: 84
Total frames with unique IPP: 28
Total overlapping frames: 28
Origin: [-55.0828, -40.1437, -57.8139]
input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.2.276.0.7230010.3.1.3.142684637

# Resampling ground truth segmentations to T2

In [57]:
!rm -rf /content/qin_prostate_root_data/resampled/nii/*
!rm -rf /content/qin_prostate_root_data/resampled/nrrd/*

In [58]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*_SEG", "*.nii.gz"), recursive=True)
for seg_nii in sorted(nii_lst) :
  print(seg_nii)
  patientID = seg_nii.split('/')[-4]
  studyUID = seg_nii.split('/')[-3]
  serieUID = seg_nii.split('/')[-2][:-4] #standard serieUID without T2_SEG custom suffix
  #get referencedSerieUID from seg corresponding to T2 MR volume 
  referencedSerieUID = str(selection_df[selection_df["SeriesInstanceUID"] == serieUID]["RefSerieUID"].iloc[0])	
  matched_mod = glob.glob(os.path.join(os.environ['qin_prostate_rep_nii'], 
                                            patientID, studyUID, referencedSerieUID + "_MR", "*.nii*"))[0]
  output_path_seg_root = os.path.join(os.environ["qin_prostate_rep_nii_resampled"],
                                  patientID, studyUID,
                                  serieUID + "_SEG")
  if not os.path.exists(output_path_seg_root):
    !mkdir -p $output_path_seg_root
  resample_args = {"input" : seg_nii, 
                        "output" : os.path.join(output_path_seg_root, seg_nii.split('/')[-1]),
                        "fixed" : matched_mod,
                        "interpolation" : "nn"}
  #resample to t2 mod based on study -- not uniform across studies
  pypla.resample(verbose = False, **resample_args)

/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/1.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/2.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/3.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.2.276.0.7230010.3.1.3.1426846371.9324.1513205197.742_SEG/1.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.2.276.0.7230010.3.1.3.1426846371.9324.1513205197.742_SEG/2.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.36

# Inference -- nnUnet

## Naming Formatting T2 files for nnUnet inference

In [59]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*T2*.nii.gz"), recursive=True)#get nii lst from data
nii_mr_t2 = [el for el in nii_lst if el.split('/')[-1].split('.')[-3].split('_')[-1] == "T2"]#get T2 lst from nii lst
for idx, el_t2 in enumerate(nii_mr_t2):
  studyInstanceUID_t2 = el_t2.split('/')[-3]
  patientID_t2 = el_t2.split("/")[-4]
  #format to nnunet 
  t2_out_nnunet = os.path.join(os.environ["nnUNet_preprocessed"], "_".join([studyInstanceUID_t2, patientID_t2, "0000"]) + ".nii.gz")
  #copy to nnUnet folder
  !cp $el_t2 $t2_out_nnunet

## Inference

In [60]:
!nnUNet_print_pretrained_model_info Task024_Promise



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Prostate MR Image Segmentation 2012. 
Segmentation target is the prostate, 
Input modalities are 0: T2. 
Also see https://promise12.grand-challenge.org/


In [61]:
print(f"input folder path : "+os.environ["nnUNet_preprocessed"])
print(f"output folder path : "+os.environ["RESULTS_FOLDER"])

input folder path : /content/nnUNet/data/processed
output folder path : /content/nnUNet/output/preds


In [62]:
!echo {os.environ["nnUNet_preprocessed"]} 

/content/nnUNet/data/processed


In [63]:
%%capture
!nnUNet_install_pretrained_model_from_zip $out_path_mod

In [71]:
!nnUNet_predict --input_folder {os.environ["nnUNet_preprocessed"]} \
                --output_folder {os.environ["RESULTS_FOLDER"]} \
                --task_name "Task024_Promise" --model $model_type \
                --save_npz 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/nnUNet/output/preds/nnUNet/3d_fullres/Task024_Promise/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 2 unique case ids, here are some examples: ['1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454_PCAMPMRI-00014'
 '1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880_PCAMPMRI-00014']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 2
number of cases that still need to be predicted: 2
emptying cuda cache
loading parameters for folds, None
folds is None so we w

# Resamplings preds back to input data

In [72]:
for pred_path in sorted(glob.glob(os.path.join(os.environ["RESULTS_FOLDER"], "*.nii.gz"))):
  search_t2_path = os.path.join(os.environ["qin_prostate_rep_nii"], pred_path.split('/')[-1].split('_')[1][:-7], 
                                   pred_path.split('/')[-1].split('_')[0], "**", 
                                   "*T2.nii.gz")
  print(f"search path for t2 : {search_t2_path}")
  t2_path = glob.glob(search_t2_path, recursive=True)[0]                              
  print(f"t2_path found : {t2_path}")
  print(f"pred path : {pred_path}")
  resample_args_to_t2_origin = {"input" : pred_path, 
                        "output" : os.path.join(os.environ["nnUNet_preds_resampled"], 
                                                pred_path.split('/')[-1]),
                        "fixed" : t2_path,
                        "interpolation" : "nn"}
  path_log = 'log_pypla_res_pred' + pred_path.split('/')[-1].split('.')[0] + '.txt'            
  !touch path_log
  pypla.resample(verbose = False, **resample_args_to_t2_origin, path_to_log_file=path_log)

search path for t2 : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/**/*T2.nii.gz
t2_path found : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172_MR/1.3.6.1.4.1.14519.5.2.1.3671.4754.15546326595350814292025022_MR_T2.nii.gz
pred path : /content/nnUNet/output/preds/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454_PCAMPMRI-00014.nii.gz
search path for t2 : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/**/*T2.nii.gz
t2_path found : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.3.6.1.4.1.14519.5.2.1.3671.4754.654327272206188613770270554127_MR/1.3.6.1.4.1.14519.5.2.1.3671.4754.65432727220618861377027055_MR_T2.nii.gz
pred path : /content/n

# Analysis

## Functions aiming to retrieve right ground truth labelsID per nnUnet prediction

In [73]:
def retrieve_seg_labelID(input_path_json, categoryCode = 'Anatomical Structure', typeCode ='Prostate'):
  #read json file
  json_dic = json.load(open(input_path_json))
  for seg in json_dic['segmentAttributes']:
    cat_code = seg[0].get('SegmentedPropertyCategoryCodeSequence').get('CodeMeaning')
    type_code = seg[0].get('SegmentedPropertyTypeCodeSequence').get('CodeMeaning')
    current_labelID = seg[0].get('labelID')
    if categoryCode == cat_code and typeCode == type_code:
      return current_labelID
  return ''

In [74]:
def get_seg_per_mod(input_path_lst):
  res_lst = []
  for nii_file in input_path_lst:
    nii_file_patientID = nii_file.split('/')[-1].split('_')[-1][:-7]
    nii_file_studyInstanceUID = nii_file.split('/')[-1].split('_')[0]
    serieUID_seg = selection_df[(selection_df.PatientID == nii_file_patientID) \
                                & (selection_df.StudyInstanceUID == nii_file_studyInstanceUID) \
                                & (selection_df.Modality == "SEG")]["SeriesInstanceUID"].iloc[0]
    json_path = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], 
                                       nii_file_patientID, 
                                       nii_file_studyInstanceUID, 
                                       serieUID_seg+"_SEG", "*.json"))[0] 
    labelID = retrieve_seg_labelID(json_path,
                                   categoryCode=ground_truth_cat_CodeMeaning,
                                   typeCode=ground_truth_type_CodeMeaning)                              
    res_lst.append(os.path.join(os.environ["qin_prostate_rep_nii"], 
                                nii_file_patientID, 
                                nii_file_studyInstanceUID, 
                                serieUID_seg+"_SEG", str(labelID)+".nii.gz"))
  return res_lst

## Dice score computation 

In [75]:
!rm -rf {os.environ["qin_prostate_rep_analysis_verbose"]}#analysis_verbose/
!mkdir -p {os.environ["qin_prostate_rep_analysis_verbose"]}

### Compute dice score between preds and ground truth and store the results into analysis folder

In [76]:
ref_path_lst = get_seg_per_mod(sorted(glob.glob(os.path.join(os.environ['nnUNet_preds_resampled'], 
                                                             "**", "*.nii.gz"), recursive=True)))
for ref_path in sorted(ref_path_lst):
  ref_path_PatientID = ref_path.split('/')[-4]
  ref_path_StudyInstanceUID = ref_path.split('/')[-3]
  test_path = glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], 
                                     '_'.join([ref_path_StudyInstanceUID, ref_path_PatientID]) + '.nii.gz'))[0]
  output_path = os.environ["qin_prostate_rep_analysis_verbose"]
  if not os.path.exists(output_path):
    !mkdir -p $output_path 
  log_path = test_path.split('/')[-1][:-7]+"_dice_score_verbose.txt"
  !echo "ref_path {ref_path}" > {os.path.join(output_path,log_path)}
  !echo "test_path {test_path}" >> {os.path.join(output_path,log_path)}
  !plastimatch dice --all $ref_path $test_path >> {os.path.join(output_path,log_path)}

### store dice score into dictionary

In [78]:
dice_dic = {}
#iterate txt logs for dice_score computation
for txt_file in sorted(glob.glob(os.path.join(os.environ["qin_prostate_rep_analysis_verbose"], "*.txt"))): 
  with open(txt_file, encoding='utf8') as f:
    #variable placeholders for txt file info -- temporary
    pholder_dice = ''
    pholder_center_mass_pred = ''
    pholder_center_mass_ref = ''
    pholder_hsdrff = ''
    pholder_ref = ''
    pholder_test = ''
    for line in f:
        if "DICE" in line:
          pholder_dice = float(line.strip().split()[-1])
        elif "ref" in line and len(line.strip().split()) > 2:
          pholder_center_mass_ref = ' '.join([line.strip().split()[-3], 
                                             line.strip().split()[-2], line.strip().split()[-1]])
        elif "cmp" in line and len(line.strip().split()) > 2:
          pholder_center_mass_pred = ' '.join([line.strip().split()[-3], 
                                             line.strip().split()[-2], line.strip().split()[-1]])                                             
        elif len(line.strip().split()) <= 5 and 'Hausdorff distance' in line:
          pholder_hsdrff = float(line.strip().split()[-1])
        elif "ref_path" in line:
          pholder_ref = str(line.strip().split()[-1]) 
        elif "test_path" in line:
          pholder_test = str(line.strip().split()[-1])
        else:
          pass
    study = txt_file.split('/')[-1][:-4].split('_')[0] #retrieve studyInstanceUID from txt file name 
    patientID = txt_file.split('/')[-1][:-4].split('_')[-4] #retrieve patientID
    # push values to dict
    dice_dic[study] = {'PatientID' : patientID,
                       'dice_score' : pholder_dice, 
                       'center_mass_pred' : pholder_center_mass_pred,
                       'center_mass_ref' : pholder_center_mass_ref,
                       'hausdorff_dist' : pholder_hsdrff,
                       'ground_truth_seg_path' : pholder_ref,
                       'pred_nnnet_path' : pholder_test}            

print('-----------------------')
print(f"average dice score : {np.average(np.array([val['dice_score'] for val in dice_dic.values()]))}")
print('-----------------------')
print()
for key,val in dice_dic.items():
  study = key
  patientID = val["PatientID"]
  dice_score = val["dice_score"]
  cm_pred = val["center_mass_pred"]
  cm_ref = val["center_mass_ref"]
  hausdorff_dist = val["hausdorff_dist"]
  gt_path = val["ground_truth_seg_path"]
  pred_path = val["pred_nnnet_path"]
  print(f"study : {study}")
  print(f"patientID : {patientID}")
  print(f"ground truth seg path : {gt_path}")
  print(f"pred seg masks nnunet path : {pred_path}")
  print(f"dice_score : {dice_score}")
  print(f"center mass ref : {cm_ref}")
  print(f"center mass pred : {cm_pred}")
  print(f"hausdorff_dist : {hausdorff_dist}")
  print()


-----------------------
average dice score : 0.7619275
-----------------------

study : 1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454
patientID : PCAMPMRI-00014
ground truth seg path : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/3.nii.gz
pred seg masks nnunet path : /content/nnUNet/output/preds_resampled/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454_PCAMPMRI-00014.nii.gz
dice_score : 0.722502
center mass ref : 15.403 29.1575 -13.3831
center mass pred : 15.0977 27.6448 -9.21346
hausdorff_dist : 13.999996

study : 1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880
patientID : PCAMPMRI-00014
ground truth seg path : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.2.276.0.7230010.3.1.3.1426846371.9324.1513205197.742_SEG/3.nii.gz
pred seg mas

In [79]:
#Save dice_score to csv
with open(os.path.join(os.environ['qin_prostate_rep_analysis'], 'analysis_metrics.csv'), 'w') as f:
    f.write("StudyInstanceUID,PatientID,dice_score,center_mass_ref,center_mass_pred,hausdorff_dist, ground_truth_seg_path, pred_nnnet_path\n")
    for key,val in dice_dic.items():
      study = str(key)
      patientID = str(val["PatientID"])
      dice_score = float(val["dice_score"])
      cm_pred = str(val["center_mass_pred"])
      cm_ref = str(val["center_mass_ref"])
      hausdorff_dist = float(val["hausdorff_dist"])
      gt_path = str(val["ground_truth_seg_path"])
      pred_path = str(val["pred_nnnet_path"])
      f.write(f"{study},{patientID},{dice_score},{cm_ref},{cm_pred},{hausdorff_dist},{gt_path},{pred_path}\n")

### All metrics

In [80]:
ref_path_lst = get_seg_per_mod(sorted(glob.glob(os.path.join(os.environ['nnUNet_preds_resampled'],                                                              "**", "*.nii.gz"), recursive=True)))
for ref_path in sorted(ref_path_lst):
  ref_path_PatientID = ref_path.split('/')[-4]#ref_path.split('/')[-1].split('_')[-1][:-7]
  ref_path_StudyInstanceUID = ref_path.split('/')[-3]#ref_path.split('/')[-1].split('_')[0]
  test_path = glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], 
                                     '_'.join([ref_path_StudyInstanceUID, ref_path_PatientID]) + '.nii.gz'))[0]
  output_path = "analysis/txt/"
  if not os.path.exists(output_path):
    !mkdir -p $output_path 
  log_path = test_path.split('/')[-1][:-7]+".txt"
  print(f"ref path : {ref_path}")
  print(f"test path : {test_path}")
  !plastimatch dice --all $ref_path $test_path 
  print()
  print()

ref path : /content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/3.nii.gz
test path : /content/nnUNet/output/preds_resampled/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454_PCAMPMRI-00014.nii.gz
CENTER_OF_MASS
ref	        15.403	       29.1575	      -13.3831
cmp	       15.0977	       27.6448	      -9.21346
TP:         83769
TN:      11413915
FN:          2443
FP:         61905
DICE:      0.722502
SE:      0.971663
SP:      0.994606
Hausdorff distance = 13.999996
Avg average Hausdorff distance = 1.313273
Max average Hausdorff distance = 2.606142
Percent (0.95) Hausdorff distance = 5.249999
Hausdorff distance (boundary) = 13.999996
Avg average Hausdorff distance (boundary) = 4.201497
Max average Hausdorff distance (boundary) = 5.490936
Percent (0.95) Hausdorff distance (boundary) = 11.845699


ref path : /content/qin_prostate_root_data/nii/PCAMPMRI-00

# EXPORT PREDS TO DCM

Get template from dropbox for whole prostate gland 

-----

try manually then automate !


---

 with resampled preds and seg gts and t2 vols


In [122]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_whole_prostate_json = "https://www.dropbox.com/s/rywyhr0wlqmhpfd/metadata_seg_nnunet_task024.json?dl=0"
seg_dcm_metadata_json_file = "pred_metadata.json"#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $seg_dcm_metadata_json_file $seg_whole_prostate_json

--2022-08-30 23:54:24--  https://www.dropbox.com/s/rywyhr0wlqmhpfd/metadata_seg_nnunet_task024.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/rywyhr0wlqmhpfd/metadata_seg_nnunet_task024.json [following]
--2022-08-30 23:54:24--  https://www.dropbox.com/s/raw/rywyhr0wlqmhpfd/metadata_seg_nnunet_task024.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2778f7e2d905f29aca721aeabc.dl.dropboxusercontent.com/cd/0/inline/BsACEjTDU6qH6_s7zp98q-ZsFTBGXABM_E2oAxlH90mFN-frdzKrP04jrpsEdYs06H-Ub_puddYhZqgX5KC3xFg7qwNqqdXYp1dnjTkDC72ovqVygSzSknRhp_AqN02b6SzxiuimzeZq5d0rrxUgTD6OsW06uT0nlcKWBupFYtwVBw/file# [following]
--2022-08-30 23:54:24--  https://uc2778f7e2d905f29aca721aeabc.dl.dropboxusercontent.com/cd/0/inline/BsACEjTDU6qH6_s

In [85]:
nii_list = sorted(glob.glob(os.path.join(nii_seg_folder, '*.nii.gz')))
for el in nii_list : print(el)

/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/1.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/2.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994_SEG/3.nii.gz


In [130]:
, output
for nii_seg_pred in glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], '*.nii.gz')):
  study_current = nii_seg_pred.split('/')[-1].split('_')[0]
  patID = nii_seg_pred.split('/')[-1].split('_')[1][:-7]

  study_mr_t2_serieUID = selection_df[(selection_df["StudyInstanceUID"] == study_current) & (selection_df["Modality"] == "MR") \
                                      & (selection_df["SeriesDescription"].str.contains('T2', na=False))]["SeriesInstanceUID"].unique()[0]
  study_seg_serieUID = selection_df[(selection_df["StudyInstanceUID"] == study_current) & (selection_df["Modality"] == "SEG") \
                                      & (selection_df["RefSerieUID"].str.contains(study_mr_t2_serieUID, na=False))]["SeriesInstanceUID"].unique()[0]
  #find t2 dcm folder
  t2_dcm_folder = os.path.join(os.environ["qin_prostate_rep_dicom"], patID, study_current, study_mr_t2_serieUID)
  #find seg dcm file
  nii_seg_folder = os.path.join(os.environ["qin_prostate_rep_dicom"], patID, study_current, study_seg_serieUID)
  # find nii seg folder == preds resampled
  assert os.path.exists(t2_dcm_folder)
  assert os.path.exists(nii_seg_folder)
  print('\nConverting...')
  print(f'pred nnunet procssed : {nii_seg_pred}')
  print(f't2_dcm_folder : {t2_dcm_folder}')
  print(f'nii_seg_folder : {nii_seg_folder}')
  output_path = os.path.join(os.environ["nnUNet_preds_resampled_dcm"], '_'.join([patID,study_current,study_seg_serieUID])+'.dcm')
  #find gt seg dcm file == orginal idc dcm files
  #convert nii pred to dcm
  !itkimage2segimage --inputImageList $nii_seg_pred \
  --inputDICOMDirectory  $t2_dcm_folder \
  --inputMetadata  $seg_dcm_metadata_json_file \
  --outputDICOM $output_path \
  --skip
  print("Done!")


Converting...
pred nnunet procssed : /content/nnUNet/output/preds_resampled/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880_PCAMPMRI-00014.nii.gz
t2_dcm_folder : /content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.3.6.1.4.1.14519.5.2.1.3671.4754.654327272206188613770270554127
nii_seg_folder : /content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.2.276.0.7230010.3.1.3.1426846371.9324.1513205197.742
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Loaded segmentation from /content/nnUNet/output/preds_resampled/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880_PCAMPMRI-00014.nii.gz
Searching recursively /content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880/1.3.6.1.4.1.14519.5.2.1.3671.4754.654327272206188613770270554127 for DICO

# EXPORT TO OHIF 

## User input 

Set these values. 

In [135]:
project_name = 'idc-sandbox-003'
location_id = 'us-central1'
my_bucket = 'nnunet_seg_test01_bucket'
my_dataset = 'nnunet_seg_test01'
my_datastore = 'qin-prost-rep_task024'

## Package setup

# GCP Healthcare steps

In order to view the MR data along with the SEGS, we need to store our data in the cloud. We first need to create a dataset, and a DICOM datastore, which will hold our MR and SEG data. To do this we will first store our data in a bucket, and then copy over the contents to the DICOM datastore. 

We first need to create a dataset if it doesn't already exist. 

In [137]:
# First let's list the datasets that we already have for our particular project_id and location_id
datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))

datasets that exist for project_id idc-sandbox-003, location us-central1: ['nnunet_seg_test01']


Create a dicom store if it doesn't already exist

In [138]:
# # First list the datastores that exist in the dataset
datastores = !gcloud healthcare dicom-stores list --project $project_name --dataset $my_dataset #--format="value(ID)"
print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(my_dataset) + ': ' + str(datastores))

datastores that exist for project_id idc-sandbox-003, location us-central1, dataset nnunet_seg_test01: ['ID                           LOCATION     TOPIC', 'nnunet_seg_test01_datastore  us-central1', 'qin-prost-rep_task024        us-central1']


Create a bucket to store the MR data and the SEG. 

In [150]:
#delete temp bucket
!gsutil -m rm -r gs://nnunet_mr_test01_bucket
#/qin-prost-rep/task024
# Create bucket to store SEG PRED -- TEST 
!gsutil -m mb -p $project_name gs://nnunet_mr_test01_bucket
#/qin-prost-rep/task024

Removing gs://nnunet_mr_test01_bucket/qin-prost-rep/task024/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.2.276.0.7230010.3.1.3.1426846371.11304.1513205200.994/1.2.276.0.7230010.3.1.4.1426846371.11304.1513205200.995.dcm#1661905240642525...
Removing gs://nnunet_mr_test01_bucket/qin-prost-rep/task024/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172/1.3.6.1.4.1.14519.5.2.1.3671.4754.112668361349339872835752848034.dcm#1661905239832042...
Removing gs://nnunet_mr_test01_bucket/qin-prost-rep/task024/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172/1.3.6.1.4.1.14519.5.2.1.3671.4754.158196449783939526819144847899.dcm#1661905239719454...
Removing gs://nnunet_mr_test01_bucket/qin-prost-rep/task024/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.1554632659535081429202

In [151]:
# Add MRI T2 and SEG to bucket 

# MRI T2 

!gsutil -m cp -r {os.environ["qin_prostate_rep_dicom"]+'/*'} gs://nnunet_mr_test01_bucket/qin-prost-rep/task024
# !gsutil -m cp -r {adc_dcm_folder+'/*'} gs://nnunet_mr_test01_bucket

# SEG - DCMQI TEST
!gsutil -m cp -r {os.environ["nnUNet_preds_resampled_dcm"]+'/*'} gs://nnunet_mr_test01_bucket/qin-prost-rep/task024
# !gsutil -m cp /content/test_gt_nrrd.dcm gs://nnunet_mr_test01_bucket

# SEG - PREDS - IDC DCM
# !gsutil -m cp $gt_seg_dcm_file gs://nnunet_mr_test01_bucket/qin-prost-rep/task024


Copying file:///content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172/1.3.6.1.4.1.14519.5.2.1.3671.4754.193576943027024210366917498868.dcm [Content-Type=application/dicom]...
Copying file:///content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172/1.3.6.1.4.1.14519.5.2.1.3671.4754.202614284921370325061189349241.dcm [Content-Type=application/dicom]...
Copying file:///content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650697728529329442454/1.3.6.1.4.1.14519.5.2.1.3671.4754.155463265953508142920250222172/1.3.6.1.4.1.14519.5.2.1.3671.4754.224756733361199424966131992139.dcm [Content-Type=application/dicom]...
Copying file:///content/qin_prostate_root_data/dicom/PCAMPMRI-00014/1.3.6.1.4.1.14519.5.2.1

Copy the MR data and SEG from bucket to the dicom datastore we created above. 

In [152]:
# Add MRI and SR from bucket to dicomstore 

!gcloud healthcare dicom-stores import gcs $my_datastore \
                                            --dataset=$my_dataset \
                                            --location=$location_id \
                                            --project=$project_name \
                                            --gcs-uri=gs://nnunet_mr_test01_bucket/qin-prost-rep/task024/**.dcm

Request issued for: [qin-prost-rep_task024]
name: projects/idc-sandbox-003/locations/us-central1/datasets/nnunet_seg_test01/dicomStores/qin-prost-rep_task024


Navigate to the OHIF app you created and open the DICOM datastore. 

# Export results to google drive 

## define input and output paths for idc_data and nnunet in google drive through colab

In [ ]:
# #path to IDC data
# input_idc_data = os.environ["qin_prostate_rep_root"]
# output_drive_idc_data = os.path.join(path_experiment, "idc_data")
# if not os.path.exists(output_drive_idc_data):
#   !mkdir -p $output_drive_idc_data
  
# #path to nnunet preds
# input_nnunet = os.environ["nnUNet"]
# output_nnunet = os.path.join(path_experiment, "nnUNet")
# if not os.path.exists(output_nnunet):
#   !mkdir -p $output_nnunet

In [ ]:
# #sanity checks regarding storage size
# ##data
# !du -sh $input_idc_data
# ##preds
# !du -sh $input_nnunet

## Copy data to drive

In [ ]:
# #copy idc data
# %cp -av $input_idc_data $output_drive_idc_data
# #verbose True, keep same struct as input
# #copy nnunet data
# %cp -av $input_nnunet $output_nnunet